# רשתות גנרטיביות

רשתות עצביות חוזרות (RNNs) והגרסאות שלהן עם תאים מבוקרים, כמו תאי זיכרון לטווח ארוך וקצר (LSTMs) ויחידות חוזרות מבוקרות (GRUs), מספקות מנגנון למידול שפה, כלומר, הן יכולות ללמוד את סדר המילים ולספק תחזיות למילה הבאה ברצף. הדבר מאפשר לנו להשתמש ב-RNNs למשימות **גנרטיביות**, כמו יצירת טקסט רגיל, תרגום מכונה ואפילו יצירת כיתוב לתמונות.

בארכיטקטורת RNN שדנו בה ביחידה הקודמת, כל יחידת RNN הפיקה את מצב החבוי הבא כתוצאה. עם זאת, ניתן גם להוסיף פלט נוסף לכל יחידה חוזרת, מה שיאפשר לנו להפיק **רצף** (ששווה באורכו לרצף המקורי). יתרה מכך, ניתן להשתמש ביחידות RNN שאינן מקבלות קלט בכל שלב, אלא רק לוקחות וקטור מצב התחלתי, ואז מפיקות רצף של פלטים.

במחברת זו, נתמקד במודלים גנרטיביים פשוטים שעוזרים לנו ליצור טקסט. לשם הפשטות, נבנה **רשת ברמת תווים**, שמייצרת טקסט אות אחר אות. במהלך האימון, נצטרך לקחת קורפוס טקסט ולחלק אותו לרצפי אותיות.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## בניית אוצר מילים של תווים

כדי לבנות רשת גנרטיבית ברמת תווים, עלינו לפצל את הטקסט לתווים בודדים במקום למילים. ניתן לעשות זאת על ידי הגדרת טוקנייזר שונה:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


בואו נראה את הדוגמה כיצד ניתן לקודד את הטקסט מתוך מערך הנתונים שלנו:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## אימון RNN גנרטיבי

הדרך שבה נאמן RNN לייצר טקסט היא כדלקמן. בכל שלב, ניקח רצף של תווים באורך `nchars`, ונבקש מהרשת לייצר את התו הבא עבור כל תו קלט:

![תמונה המציגה דוגמה ליצירת המילה 'HELLO' באמצעות RNN.](../../../../../translated_images/he/rnn-generate.56c54afb52f9781d.webp)

בהתאם לתרחיש בפועל, ייתכן שנרצה לכלול גם תווים מיוחדים, כמו *סוף רצף* `<eos>`. במקרה שלנו, אנחנו רק רוצים לאמן את הרשת ליצירת טקסט אינסופי, ולכן נקבע את גודל כל רצף להיות שווה ל-`nchars` טוקנים. כתוצאה מכך, כל דוגמת אימון תכלול `nchars` קלטים ו-`nchars` פלטים (שהם רצף הקלט מוזז סמל אחד שמאלה). מיניבאץ' יכלול כמה רצפים כאלה.

הדרך שבה נייצר מיניבאצ'ים היא לקחת כל טקסט חדשות באורך `l`, ולייצר ממנו את כל שילובי הקלט-פלט האפשריים (יהיו `l-nchars` שילובים כאלה). הם יהוו מיניבאץ' אחד, וגודל המיניבאצ'ים יהיה שונה בכל שלב אימון.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

עכשיו נגדיר את רשת הגנרטור. היא יכולה להתבסס על כל תא חוזר שדיברנו עליו ביחידה הקודמת (פשוט, LSTM או GRU). בדוגמה שלנו נשתמש ב-LSTM.

מכיוון שהרשת מקבלת תווים כקלט וגודל אוצר המילים די קטן, אין צורך בשכבת הטמעה; קלט מקודד ב-one-hot יכול לעבור ישירות לתא LSTM. עם זאת, מכיוון שאנחנו מעבירים מספרי תווים כקלט, עלינו לקודד אותם ב-one-hot לפני העברתם ל-LSTM. זה נעשה על ידי קריאה לפונקציה `one_hot` במהלך מעבר `forward`. מקודד הפלט יהיה שכבה ליניארית שתמיר את המצב הנסתר לפלט מקודד ב-one-hot.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

במהלך האימון, אנחנו רוצים להיות מסוגלים לדגום טקסט שנוצר. כדי לעשות זאת, נגדיר פונקציה בשם `generate` שתפיק מחרוזת פלט באורך `size`, שמתחילה מהמחרוזת ההתחלתית `start`.

כך זה עובד: ראשית, נעביר את כל המחרוזת ההתחלתית דרך הרשת, ונקבל את מצב הפלט `s` ואת התו הבא החזוי `out`. מכיוון ש-`out` מקודד בשיטת one-hot, נשתמש ב-`argmax` כדי לקבל את האינדקס של התו `nc` במילון, ונשתמש ב-`itos` כדי לזהות את התו בפועל ולהוסיף אותו לרשימת התווים `chars`. התהליך הזה של יצירת תו אחד חוזר על עצמו `size` פעמים כדי לייצר את מספר התווים הנדרש.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

עכשיו בואו נתחיל את האימון! לולאת האימון כמעט זהה לכל הדוגמאות הקודמות שלנו, אבל במקום דיוק אנחנו מדפיסים טקסט שנוצר באופן מדגמי כל 1000 אפוקים.

יש לשים דגש מיוחד על הדרך שבה אנחנו מחשבים את ההפסד. אנחנו צריכים לחשב את ההפסד בהתחשב בפלט המקודד ב-one-hot `out`, ובטקסט הצפוי `text_out`, שהוא רשימת אינדקסי התווים. למרבה המזל, הפונקציה `cross_entropy` מצפה לפלט לא מנורמל של הרשת כארגומנט הראשון, ולמספר המחלקה כארגומנט השני, שזה בדיוק מה שיש לנו. היא גם מבצעת ממוצע אוטומטי על גודל המיני-באטץ'.

אנחנו גם מגבילים את האימון לפי מספר הדגימות `samples_to_train`, כדי שלא נצטרך להמתין יותר מדי זמן. אנחנו מעודדים אתכם להתנסות ולנסות אימון ארוך יותר, אולי לכמה אפוקים (במקרה כזה תצטרכו ליצור לולאה נוספת סביב הקוד הזה).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

דוגמה זו כבר מייצרת טקסט די טוב, אך ניתן לשפר אותה בכמה דרכים:

* **שיפור יצירת מיניבאצ'ים**. הדרך שבה הכנו את הנתונים לאימון הייתה יצירת מיניבאץ' אחד מתוך דגימה אחת. זה לא אידיאלי, מכיוון שמיניבאצ'ים הם בגדלים שונים, וחלקם אפילו לא יכולים להיווצר, כי הטקסט קטן מ-`nchars`. בנוסף, מיניבאצ'ים קטנים לא מנצלים את ה-GPU בצורה מספקת. יהיה חכם יותר לקחת מקטע טקסט גדול מכל הדגימות, ואז ליצור את כל זוגות הקלט-פלט, לערבב אותם, וליצור מיניבאצ'ים בגודל שווה.

* **LSTM רב-שכבתי**. יש היגיון לנסות 2 או 3 שכבות של תאי LSTM. כפי שציינו ביחידה הקודמת, כל שכבה של LSTM מוציאה דפוסים מסוימים מהטקסט, ובמקרה של מחולל ברמת תווים, ניתן לצפות שהשכבה הנמוכה של LSTM תהיה אחראית על חילוץ הברות, והשכבות הגבוהות - על מילים ושילובי מילים. ניתן ליישם זאת בפשטות על ידי העברת פרמטר מספר-השכבות לבנאי של LSTM.

* ייתכן שתרצה גם להתנסות עם **יחידות GRU** ולבדוק אילו מהן מבצעות טוב יותר, וכן עם **גדלים שונים של שכבות נסתרות**. שכבה נסתרת גדולה מדי עשויה להוביל לבעיה של התאמת יתר (לדוגמה, הרשת תלמד את הטקסט המדויק), וגודל קטן יותר עשוי שלא להפיק תוצאה טובה.


## יצירת טקסט רך וטמפרטורה

בהגדרה הקודמת של `generate`, תמיד בחרנו את התו עם ההסתברות הגבוהה ביותר כתו הבא בטקסט שנוצר. הדבר הוביל לכך שהטקסט לעיתים קרובות "חזר על עצמו" בין רצפי תווים זהים שוב ושוב, כמו בדוגמה הזו:
```
today of the second the company and a second the company ...
```

עם זאת, אם נבחן את התפלגות ההסתברויות עבור התו הבא, ייתכן שההבדל בין כמה מההסתברויות הגבוהות ביותר אינו גדול, לדוגמה, תו אחד יכול להיות בעל הסתברות של 0.2, ותו אחר - 0.19, וכו'. לדוגמה, כאשר מחפשים את התו הבא ברצף '*play*', התו הבא יכול להיות באותה מידה רווח או **e** (כמו במילה *player*).

מסקנה זו מובילה אותנו להבנה שלא תמיד "הוגן" לבחור את התו עם ההסתברות הגבוהה ביותר, משום שבחירה בתו עם ההסתברות השנייה הגבוהה עדיין יכולה להוביל לטקסט משמעותי. חכם יותר **לדגום** תווים מתוך התפלגות ההסתברויות שמתקבלת מפלט הרשת.

דגימה זו יכולה להתבצע באמצעות הפונקציה `multinomial`, שמיישמת את מה שנקרא **התפלגות מולטינומית**. פונקציה שמיישמת את יצירת הטקסט ה**רך** הזו מוגדרת להלן:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

הוספנו פרמטר נוסף שנקרא **טמפרטורה**, שמשמש לציון עד כמה עלינו להיצמד להסתברות הגבוהה ביותר. אם הטמפרטורה היא 1.0, אנו מבצעים דגימה מולטינומית הוגנת, וכאשר הטמפרטורה עולה לאינסוף - כל ההסתברויות הופכות לשוות, ואנו בוחרים באופן אקראי את התו הבא. בדוגמה למטה ניתן לראות שהטקסט הופך לחסר משמעות כאשר אנו מעלים את הטמפרטורה יותר מדי, והוא מזכיר טקסט "מחזורי" שנוצר באופן קשיח כאשר הטמפרטורה מתקרבת ל-0.



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
